## Načtení knihoven

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import random
import glob
from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers , callbacks
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import tensorflow as tf
import os
from tqdm import tqdm
import cv2
from keras.preprocessing.image import img_to_array
from sklearn import metrics
import itertools
from sklearn.metrics import confusion_matrix , classification_report
from sklearn.neighbors import KernelDensity

## Normalizace a načtení dat

In [4]:
train_dir = '../input/screw-anomaly-dataset/screw_dataset/AE_dataset/train'
test_dir = '../input/screw-anomaly-dataset/screw_dataset/AE_dataset/test'
anomaly_dir = '../input/screw-anomaly-dataset/screw_dataset/AE_dataset/anomaly'

In [5]:
batch_size = 16
image_size = (256,256)

normalize = ImageDataGenerator(rescale=1./255)

train_ds = normalize.flow_from_directory(
    train_dir,
    target_size = image_size,
    batch_size = batch_size,
    shuffle=True,
    seed=3,
    class_mode = 'input'
    )

## Autoencoder

In [6]:
# Encoder
model = Sequential()
model.add(Conv2D(32,(3,3), activation='relu', padding='same', input_shape=(256,256,3)))
model.add(MaxPooling2D((2,2), padding='same'))
model.add(Conv2D(64,(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D((2,2), padding='same'))
model.add(Conv2D(128,(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D((2,2), padding='same'))
model.add(Conv2D(256,(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D((2,2), padding='same'))
model.add(Conv2D(512,(3,3), activation='relu', padding='same'))
model.add(MaxPooling2D((2,2)))

# Decoder
model.add(Conv2D(512,(3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(256,(3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(128,(3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(64,(3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32,(3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2,2)))

model.add(Conv2D(3,(3,3), activation='sigmoid', padding='same'))

In [7]:
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mse'])

model.summary()

## Train model

In [ ]:
history = model.fit(train_ds,
                    batch_size=16,
                    shuffle=True,
                    verbose=1,
                    epochs=300)

model.save("AE.h5")

In [7]:
models_dir = '../input/modelsae/AE.h5'
model = tf.keras.models.load_model(models_dir)

In [14]:
encoder = Sequential()
encoder.add(Conv2D(32,(3,3), activation='relu', padding='same', input_shape=( 256, 256, 3), weights=model.layers[0].get_weights()) )
encoder.add(MaxPooling2D((2, 2), padding='same'))
encoder.add(Conv2D(64,(3,3), activation='relu', padding='same', weights=model.layers[2].get_weights()))
encoder.add(MaxPooling2D((2, 2), padding='same'))
encoder.add(Conv2D(128,(3,3), activation='relu', padding='same', weights=model.layers[4].get_weights()))
encoder.add(MaxPooling2D((2, 2), padding='same'))
encoder.add(Conv2D(256,(3,3), activation='relu', padding='same', weights=model.layers[6].get_weights()))
encoder.add(MaxPooling2D((2, 2), padding='same'))
encoder.add(Conv2D(512,(3,3), activation='relu', padding='same', weights=model.layers[8].get_weights()))

encoder.summary()

In [8]:
normilize = tf.keras.Sequential([layers.Rescaling(1./255)])

In [9]:
good_data = []
test_files = glob.glob('../input/screw-anomaly-dataset/screw_dataset/AE_dataset/test/good/*')
for i in tqdm(test_files[0:19]):
    img = cv2.imread(i,1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(256,256))
    good_data.append(img)

good_data = np.reshape(good_data, (len(good_data), 256, 256, 3))
good_data = normilize.predict(good_data)

In [10]:
anomaly_data = []
anomaly_files = glob.glob('../input/screw-anomaly-dataset/screw_dataset/AE_dataset/anomaly/*/*')
for i in tqdm(anomaly_files[0:45]):
    img = cv2.imread(i,1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(256,256))
    anomaly_data.append(img)
    
anomaly_data = np.reshape(anomaly_data, (len(anomaly_data), 256, 256, 3))
anomaly_data = normilize.predict(anomaly_data)

In [11]:
test_good_data = []
test_good_files = glob.glob('../input/screw-anomaly-dataset/screw_dataset/AE_dataset/test/good/*')
for i in tqdm(test_good_files[20:42]):
    img = cv2.imread(i,1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(256,256))
    test_good_data.append(img)

test_good_data = np.reshape(test_good_data, (len(test_good_data), 256, 256, 3))
test_good_data = normilize.predict(test_good_data)

In [12]:
test_anomaly_data = []
test_anomaly_files = glob.glob('../input/screw-anomaly-dataset/screw_dataset/AE_dataset/anomaly/*/*')
for i in tqdm(test_anomaly_files[46:120]):
    img = cv2.imread(i,1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(256,256))
    test_anomaly_data.append(img)
    
test_anomaly_data = np.reshape(test_anomaly_data, (len(test_anomaly_data), 256, 256, 3))
test_anomaly_data = normilize.predict(test_anomaly_data)

In [15]:
encoded_images = encoder.predict_generator(train_ds)
encoder_output_shape = encoder.output_shape
out_vector_shape = encoder_output_shape[1]*encoder_output_shape[2]*encoder_output_shape[3]
encoded_images_vector = [np.reshape(img, (out_vector_shape)) for img in encoded_images]

In [16]:
ref_density = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(encoded_images_vector)

In [17]:
def density_and_recon_err(batch_images):
    density_list=[]
    recon_error_list=[]
    for im in tqdm(range(0, batch_images.shape[0]-1)):
        
        img  = batch_images[im]
        img = img[np.newaxis, :,:,:]
        encoded_img = encoder.predict([[img]],verbose=0)
        encoded_img = [np.reshape(img, (out_vector_shape)) for img in encoded_img]
        density = ref_density.score_samples(encoded_img)[0]
        reconstruction = model.predict([[img]],verbose=0)
        reconstruction_error = model.evaluate([reconstruction],[[img]], batch_size = 1,verbose=0)[0]
        density_list.append(density)
        recon_error_list.append(reconstruction_error)
        
    average_density = np.mean(np.array(density_list)) 
    stdev_density = np.std(np.array(density_list))
    average_recon_error = np.mean(np.array(recon_error_list))
    stdev_recon_error = np.std(np.array(recon_error_list))
    
    return density_list,average_density,stdev_density,recon_error_list,average_recon_error,stdev_recon_error

In [18]:
test_values = density_and_recon_err(good_data)
anomaly_values = density_and_recon_err(anomaly_data)


In [19]:
print("Test Avg.Density = " + str(test_values[1]) + "(" + str(test_values[2]) +")" )
print("Test Rec.Error = " + str(test_values[4]) + "(" + str(test_values[5]) +")" )
print("Anomaly Avg.Density = " + str(anomaly_values[1]) + "(" + str(anomaly_values[2]) +")" )
print("Anomaly Rec.Error = " + str(anomaly_values[4]) + "(" + str(anomaly_values[5]) +")" )

In [20]:
fig, ax = plt.subplots(figsize=(8, 15))
ax.set_aspect(5000)

x1 = test_values[0]
y1 = test_values[3]
x2 = anomaly_values[0]
y2 = anomaly_values[3]

ax.scatter(x1, y1, color="g",label='Snímky bez vady')
ax.scatter(x2, y2, color="r", label= 'Snímky s vadou')


ax.set_xlim(90482.5,90500)
ax.set_ylim(0.0008, 0.0028)
plt.xlabel("Odhad hustoty",fontsize=12)
plt.ylabel("Rekonstrkční chyba",fontsize=12)
plt.legend()

plt.show()

In [21]:
def evaluation(batch_images,type):
    predict = []
    truth = []
    density_max = 90496
    reconstruction_error_max = 0.00148
    
    for im in tqdm(range(0, batch_images.shape[0])):
        img  = batch_images[im]
        img = img[np.newaxis, :,:,:]
        
        encoded_img = encoder.predict([[img]],verbose=0) 
        encoded_img = [np.reshape(img, (out_vector_shape)) for img in encoded_img] 
        density = ref_density.score_samples(encoded_img)[0] 
        
        reconstruction = model.predict([[img]],verbose=0)
        reconstruction_error = model.evaluate([reconstruction],[[img]], batch_size = 1,verbose=0)[0]
    
        if density < density_max and reconstruction_error < reconstruction_error_max:
            predict.append(['good'])
            truth.append(type)
        else:
            predict.append(['anomaly'])
            truth.append(type)
    prediction=[truth,predict]
    return prediction

anomaly_ev = evaluation(test_anomaly_data, ['anomaly'])
test_ev = evaluation(test_good_data, ['good'])
total_ev = [anomaly_ev[0] + test_ev[0], anomaly_ev[1] + test_ev[1]]

In [22]:
image_true = total_ev[0]
image_predict = total_ev[1]
cm = confusion_matrix(y_true=image_true, y_pred=image_predict)

def plot_confusion_matrix(cm,
                          classes,
                          cmap=plt.cm.Reds):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Skutečnost',fontsize=15)
    plt.xlabel('Predikce',fontsize=15)

plot_confusion_matrix(cm=cm,classes = ['S vadou','Bez vady'])
print(classification_report(image_true, image_predict, target_names=['S vadou','Bez vady']))